# Conversational Interface - Chatbot with Claude LLM

> *This notebook should work well with the **`Data Science 3.0`** kernel in SageMaker Studio*

In this notebook, we will build a chatbot using the Foundation Models (FMs) in Amazon Bedrock. For our use-case we use Claude as our FM for building the chatbot.

## Overview

Conversational interfaces such as chatbots and virtual assistants can be used to enhance the user experience for your customers.Chatbots uses natural language processing (NLP) and machine learning algorithms to understand and respond to user queries. Chatbots can be used in a variety of applications, such as customer service, sales, and e-commerce, to provide quick and efficient responses to users. They can be accessed through various channels such as websites, social media platforms, and messaging apps.


## Chatbot using Amazon Bedrock

![Amazon Bedrock - Conversational Interface](./images/chatbot_bedrock.png)


## Use Cases

1. **Chatbot (Basic)** - Zero Shot chatbot with a FM model
2. **Chatbot using prompt** - template(Langchain) - Chatbot with some context provided in the prompt template
3. **Chatbot with persona** - Chatbot with defined roles. i.e. Career Coach and Human interactions
4. **Contextual-aware chatbot** - Passing in context through an external file by generating embeddings.

## Langchain framework for building Chatbot with Amazon Bedrock
In Conversational interfaces such as chatbots, it is highly important to remember previous interactions, both at a short term but also at a long term level.

LangChain provides memory components in two forms. First, LangChain provides helper utilities for managing and manipulating previous chat messages. These are designed to be modular and useful regardless of how they are used. Secondly, LangChain provides easy ways to incorporate these utilities into chains.
It allows us to easily define and interact with different types of abstractions, which make it easy to build powerful chatbots.

## Building Chatbot with Context - Key Elements

The first process in a building a contextual-aware chatbot is to **generate embeddings** for the context. Typically, you will have an ingestion process which will run through your embedding model and generate the embeddings which will be stored in a sort of a vector store. In this example we are using Titan Embeddings model for this

![Embeddings](./images/embeddings_lang.png)

Second process is the user request orchestration , interaction,  invoking and returing the results

![Chatbot](./images/chatbot_lang.png)

## Architecture [Context Aware Chatbot]
![4](./images/context-aware-chatbot.png)


## Setup

⚠️ ⚠️ ⚠️ Before running this notebook, ensure you've run the [Bedrock boto3 setup notebook](../00_Intro/bedrock_boto3_setup.ipynb#Prerequisites) notebook. ⚠️ ⚠️ ⚠️


In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
import json
import os
import sys

import boto3

module_path = ".."
sys.path.append(os.path.abspath(module_path))
from utils import bedrock, print_ww


# ---- ⚠️ Un-comment and edit the below lines as needed for your AWS setup ⚠️ ----

# os.environ["AWS_DEFAULT_REGION"] = "<REGION_NAME>"  # E.g. "us-east-1"
# os.environ["AWS_PROFILE"] = "<YOUR_PROFILE>"
# os.environ["BEDROCK_ASSUME_ROLE"] = "<YOUR_ROLE_ARN>"  # E.g. "arn:aws:..."


boto3_bedrock = bedrock.get_bedrock_client(
    assumed_role=os.environ.get("BEDROCK_ASSUME_ROLE", None),
    region=os.environ.get("AWS_DEFAULT_REGION", None)
)

Create new client
  Using region: us-east-1
boto3 Bedrock client successfully created!
bedrock-runtime(https://bedrock-runtime.us-east-1.amazonaws.com)


## Chatbot (Basic - without context)

We use [CoversationChain](https://python.langchain.com/en/latest/modules/models/llms/integrations/bedrock.html?highlight=ConversationChain#using-in-a-conversation-chain) from LangChain to start the conversation. We also use the [ConversationBufferMemory](https://python.langchain.com/en/latest/modules/memory/types/buffer.html) for storing the messages. We can also get the history as a list of messages (this is very useful in a chat model).

Chatbots needs to remember the previous interactions. Conversational memory allows us to do that. There are several ways that we can implement conversational memory. In the context of LangChain, they are all built on top of the ConversationChain.

**Note:** The model outputs are non-deterministic

In [4]:
from langchain.chains import ConversationChain
from langchain.llms.bedrock import Bedrock
from langchain.memory import ConversationBufferMemory
modelId = "anthropic.claude-v2"
cl_llm = Bedrock(
    model_id=modelId,
    client=boto3_bedrock,
    model_kwargs={"max_tokens_to_sample": 1000},
)
memory = ConversationBufferMemory()
conversation = ConversationChain(
    llm=cl_llm, verbose=True, memory=memory
)

try:
    
    print_ww(conversation.predict(input="Hi there!"))

except ValueError as error:
    if  "AccessDeniedException" in str(error):
        print(f"\x1b[41m{error}\
        \nTo troubeshoot this issue please refer to the following resources.\
         \nhttps://docs.aws.amazon.com/IAM/latest/UserGuide/troubleshoot_access-denied.html\
         \nhttps://docs.aws.amazon.com/bedrock/latest/userguide/security-iam.html\x1b[0m\n")      
        class StopExecution(ValueError):
            def _render_traceback_(self):
                pass
        raise StopExecution        
    else:
        raise error



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Hi there!
AI:

> Finished chain.
 Hello! Nice to meet you. What would you like to chat about today? I'm happy to have a friendly
conversation about any topics that interest you.


What happens here? We said "Hi there!" and the model spat out a several conversations. This is due to the fact that the default prompt used by Langchain ConversationChain is not well designed for Claude. An [effective Claude prompt](https://docs.anthropic.com/claude/docs/introduction-to-prompt-design) should contain `\n\nHuman:` at the beginning and also contain `\n\nAssistant:` in the prompt sometime after the `\n\nHuman:` (optionally followed by other text that you want to [put in Claude's mouth](https://docs.anthropic.com/claude/docs/human-and-assistant-formatting#use-human-and-assistant-to-put-words-in-claudes-mouth)). Let's fix this.

To learn more about how to write prompts for Claude, check [Anthropic documentation](https://docs.anthropic.com/claude/docs/introduction-to-prompt-design).

## Chatbot using prompt template (Langchain)

LangChain provides several classes and functions to make constructing and working with prompts easy. We are going to use the [PromptTemplate](https://python.langchain.com/en/latest/modules/prompts/getting_started.html) class to construct the prompt from a f-string template. 

In [5]:
from langchain.memory import ConversationBufferMemory
from langchain.prompts import PromptTemplate

# turn verbose to true to see the full logs and documents
conversation= ConversationChain(
    llm=cl_llm, verbose=False, memory=ConversationBufferMemory() #memory_chain
)

# langchain prompts do not always work with all the models. This prompt is tuned for Claude
claude_prompt = PromptTemplate.from_template("""

Human: The following is a friendly conversation between a human and an AI.
The AI is talkative and provides lots of specific details from its context. If the AI does not know
the answer to a question, it truthfully says it does not know.

Current conversation:
<conversation_history>
{history}
</conversation_history>

Here is the human's next reply:
<human_reply>
{input}
</human_reply>

Assistant:
""")

conversation.prompt = claude_prompt

print_ww(conversation.predict(input="Hi there!"))

 Hi there! Nice to meet you. I'm Claude, an AI assistant created by Anthropic to be helpful,
harmless, and honest. How are you doing today?


#### New Questions

Model has responded with intial message, let's ask few questions

In [6]:
print_ww(conversation.predict(input="Give me a few tips on how to start a new garden."))

 Here are a few tips to help you start a new garden:

- Choose a sunny spot. Most vegetables and fruits need at least 6 hours of direct sunlight per day.
Make sure to pick a spot that gets sunlight for most of the day.

- Prepare the soil. Remove any weeds or grass in the area, then mix in compost or other organic
matter to enrich the soil. Good soil is key to a productive garden. You may need to get a soil test
done to see if you need to add any amendments.

- Decide what to grow. Pick vegetables and fruits that you enjoy eating and that are suitable for
your climate. Some good options for beginners are tomatoes, peppers, leafy greens, beans, carrots
and radishes.

- Make a planting plan. Figure out how much space to leave between plants and between rows. Follow
seed packet instructions so you don't plant things too close together. Use trellises and cages where
needed.

- Plant your seeds or seedlings. Follow the planting directions on seed packets. If using
transplants, carefully rem

#### Build on the questions

Let's ask a question without mentioning the word garden to see if model can understand previous conversation

In [7]:
print_ww(conversation.predict(input="Cool. Will that work with tomatoes?"))

 Yes, the tips I provided will work great for growing tomatoes in a new garden! Here are some more
specific tips for tomatoes:

- Choose tomato varieties suited to your climate. Look for early, midseason and late varieties to
extend the harvest. Go for compact/determinate types if space is limited.

- Give tomatoes lots of sun - at least 6 hours, preferably 8-10 if possible. Sun is crucial for good
fruit production.

- Prepare the soil well with compost and other organic material. Tomatoes need nutrient-rich soil.

- Space tomato plants at least 2-3 feet apart. Give indeterminate vines even more space. Use cages
or trellises to keep plants upright.

- Plant tomatoes deeply, burying much of the stem. This encourages roots to grow along the buried
stem for a stronger, more stable plant.

- Water tomatoes regularly, about 1-2 inches per week. Irregular watering can cause problems like
cracking or blossom end rot.

- Use mulch around the bases to retain moisture. Straw or leaf mulch works 

#### Finishing this conversation

In [8]:
print_ww(conversation.predict(input="That's all, thank you!"))

 You're very welcome! I'm glad I could provide some helpful tips for starting a new garden,
especially focused on growing tomatoes. Best of luck with your gardening this season! Feel free to
reach out if you need any other advice as you get your garden going. Happy planting!


Claude is still really talkative. Try changing the prompt to make Claude provide shorter answers.

### Interactive session using ipywidgets

The following utility class allows us to interact with Claude in a more natural way. We write out the question in an input box, and get Claude's answer. We can then continue our conversation.

In [9]:
import ipywidgets as ipw
from IPython.display import display, clear_output

class ChatUX:
    """ A chat UX using IPWidgets
    """
    def __init__(self, qa, retrievalChain = False):
        self.qa = qa
        self.name = None
        self.b=None
        self.retrievalChain = retrievalChain
        self.out = ipw.Output()


    def start_chat(self):
        print("Starting chat bot")
        display(self.out)
        self.chat(None)


    def chat(self, _):
        if self.name is None:
            prompt = ""
        else: 
            prompt = self.name.value
        if 'q' == prompt or 'quit' == prompt or 'Q' == prompt:
            print("Thank you , that was a nice chat!!")
            return
        elif len(prompt) > 0:
            with self.out:
                thinking = ipw.Label(value="Thinking...")
                display(thinking)
                try:
                    if self.retrievalChain:
                        result = self.qa.run({'question': prompt })
                    else:
                        result = self.qa.run({'input': prompt }) #, 'history':chat_history})
                except:
                    result = "No answer"
                thinking.value=""
                print_ww(f"AI:{result}")
                self.name.disabled = True
                self.b.disabled = True
                self.name = None

        if self.name is None:
            with self.out:
                self.name = ipw.Text(description="You:", placeholder='q to quit')
                self.b = ipw.Button(description="Send")
                self.b.on_click(self.chat)
                display(ipw.Box(children=(self.name, self.b)))

Let's start a chat. You can also test the following questions:
1. tell me a joke
2. tell me another joke
3. what was the first joke about
4. can you make another joke on the same topic of the first joke

In [10]:
chat = ChatUX(conversation)
chat.start_chat()

Starting chat bot


Output()

## Chatbot with persona

AI assistant will play the role of a career coach. Role Play Dialogue requires user message to be set in before starting the chat. ConversationBufferMemory is used to pre-populate the dialog

In [11]:
# store previous interactions using ConversationalBufferMemory and add custom prompts to the chat.
memory = ConversationBufferMemory()
memory.chat_memory.add_user_message("You will be acting as a career coach. Your goal is to give career advice to users")
memory.chat_memory.add_ai_message("I am a career coach and give career advice")
cl_llm = Bedrock(model_id="anthropic.claude-v2",client=boto3_bedrock)
conversation = ConversationChain(
     llm=cl_llm, verbose=True, memory=memory
)

conversation.prompt = claude_prompt

print_ww(conversation.predict(input="What are the career options in AI?"))



> Entering new ConversationChain chain...
Prompt after formatting:


Human: The following is a friendly conversation between a human and an AI.
The AI is talkative and provides lots of specific details from its context. If the AI does not know
the answer to a question, it truthfully says it does not know.

Current conversation:
<conversation_history>
Human: You will be acting as a career coach. Your goal is to give career advice to users
AI: I am a career coach and give career advice
</conversation_history>

Here is the human's next reply:
<human_reply>
What are the career options in AI?
</human_reply>

Assistant:


> Finished chain.
 Here are some potential career options in the field of artificial intelligence (AI):

- Machine Learning Engineer - Develops and optimizes machine learning algorithms and models to solve
problems like computer vision, speech recognition, and predictive analytics. Requires skills in
programming languages like Python and frameworks like TensorFlow.

- AI 

In [12]:
print_ww(conversation.predict(input="What these people really do? Is it fun?"))



> Entering new ConversationChain chain...
Prompt after formatting:


Human: The following is a friendly conversation between a human and an AI.
The AI is talkative and provides lots of specific details from its context. If the AI does not know
the answer to a question, it truthfully says it does not know.

Current conversation:
<conversation_history>
Human: You will be acting as a career coach. Your goal is to give career advice to users
AI: I am a career coach and give career advice
Human: What are the career options in AI?
AI:  Here are some potential career options in the field of artificial intelligence (AI):

- Machine Learning Engineer - Develops and optimizes machine learning algorithms and models to solve problems like computer vision, speech recognition, and predictive analytics. Requires skills in programming languages like Python and frameworks like TensorFlow.

- AI Researcher - Conducts research to advance the field of AI. Works on innovating new machine learning techniq

##### Let's ask a question that is not specialty of this Persona and the model shouldn't answer that question and give a reason for that

In [13]:
conversation.verbose = False
print_ww(conversation.predict(input="How to fix my car?"))

 I apologize, but I do not have expertise in auto repair. As a career coach AI, I can provide
guidance on jobs and careers in artificial intelligence and technology fields, but I do not have
specific knowledge about fixing cars. Perhaps consulting an auto mechanic or looking up
instructional guides online could help with troubleshooting car issues. Let me know if you have any
other career-related questions I may be able to assist with!


## Chatbot with Context 
In this use case we will ask the Chatbot to answer question from some external corpus it has likely never seen before. To do this we apply a pattern called RAG (Retrieval Augmented Generation): the idea is to index the corpus in chunks, then look up which sections of the corpus might be relevant to provide an answer by using semantic similarity between the chunks and the question. Finally the most relevant chunks are aggregated and passed as context to the ConversationChain, similar to providing a history.

We will take a csv file and use **Titan Embeddings Model** to create vectors for each line of the csv. This vector is then stored in FAISS, an open source library providing an in-memory vector datastore. When the chatbot is asked a question, we query FAISS with the question and retrieve the text which is semantically closest. This will be our answer. 

#### Titan embeddings Model

Embeddings are a way to represent words, phrases or any other discrete items as vectors in a continuous vector space. This allows machine learning models to perform mathematical operations on these representations and capture semantic relationships between them.

Embeddings are for example used for the RAG [document search capability](https://labelbox.com/blog/how-vector-similarity-search-works/) 


In [14]:
from langchain.embeddings import BedrockEmbeddings

br_embeddings = BedrockEmbeddings(model_id="amazon.titan-embed-text-v1", client=boto3_bedrock)

#### FAISS as VectorStore

In order to be able to use embeddings for search, we need a store that can efficiently perform vector similarity searches. In this notebook we use FAISS, which is an in memory store. For permanently store vectors, one can use pgVector, Pinecone or Chroma.

The langchain VectorStore API's are available [here](https://python.langchain.com/en/harrison-docs-refactor-3-24/reference/modules/vectorstore.html)

To know more about the FAISS vector store please refer to this [document](https://arxiv.org/pdf/1702.08734.pdf).

In [15]:
from langchain.document_loaders import CSVLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.indexes.vectorstore import VectorStoreIndexWrapper
from langchain.vectorstores import FAISS

s3_path = "s3://jumpstart-cache-prod-us-east-2/training-datasets/Amazon_SageMaker_FAQs/Amazon_SageMaker_FAQs.csv"
!aws s3 cp $s3_path ./rag_data/Amazon_SageMaker_FAQs.csv

loader = CSVLoader("./rag_data/Amazon_SageMaker_FAQs.csv") # --- > 219 docs with 400 chars, each row consists in a question column and an answer column
documents_aws = loader.load() #
print(f"Number of documents={len(documents_aws)}")

docs = CharacterTextSplitter(chunk_size=2000, chunk_overlap=400, separator=",").split_documents(documents_aws)

print(f"Number of documents after split and chunking={len(docs)}")
vectorstore_faiss_aws = None
try:
    
    vectorstore_faiss_aws = FAISS.from_documents(
        documents=docs,
        embedding = br_embeddings
    )

    print(f"vectorstore_faiss_aws: number of elements in the index={vectorstore_faiss_aws.index.ntotal}::")

except ValueError as error:
    if  "AccessDeniedException" in str(error):
        print(f"\x1b[41m{error}\
        \nTo troubeshoot this issue please refer to the following resources.\
         \nhttps://docs.aws.amazon.com/IAM/latest/UserGuide/troubleshoot_access-denied.html\
         \nhttps://docs.aws.amazon.com/bedrock/latest/userguide/security-iam.html\x1b[0m\n")      
        class StopExecution(ValueError):
            def _render_traceback_(self):
                pass
        raise StopExecution        
    else:
        raise error

download: s3://jumpstart-cache-prod-us-east-2/training-datasets/Amazon_SageMaker_FAQs/Amazon_SageMaker_FAQs.csv to rag_data/Amazon_SageMaker_FAQs.csv
Number of documents=153
Number of documents after split and chunking=154
vectorstore_faiss_aws: number of elements in the index=154::


#### Semantic search

We can use a Wrapper class provided by LangChain to query the vector data base store and return to us the relevant documents. Behind the scenes this is only going to run a RetrievalQA chain.

In [16]:
wrapper_store_faiss = VectorStoreIndexWrapper(vectorstore=vectorstore_faiss_aws)
print_ww(wrapper_store_faiss.query("R in SageMaker", llm=cl_llm))

 Based on the provided context, here are some key points about using R with Amazon SageMaker:

- R is supported in SageMaker through RStudio Workbench and SageMaker notebook instances with R
kernels. You can use R to build, train, tune, and deploy ML models.

- Reticulate provides an R interface to the SageMaker Python SDK, enabling you to access SageMaker
functionality from R.

- RStudio on SageMaker provides a fully managed IDE for R that synchronizes with SageMaker Studio
notebooks.

- You can seamlessly switch between RStudio and SageMaker Studio notebooks to build models in R and
Python.

- All your work including code, data, and models is synchronized between RStudio and SageMaker
Studio environments.

- SageMaker Studio provides a unified visual interface to perform end-to-end ML development in R or
Python, including experiment tracking with SageMaker Experiments.

In summary, R is a fully supported language in SageMaker with features like RStudio integration,
reticulate library

Let's see how the semantic search works:
1. First we calculate the embeddings vector for the query, and
2. then we use this vector to do a similarity search on the store

In [17]:
v = br_embeddings.embed_query("R in SageMaker")
print(v[0:10])
results = vectorstore_faiss_aws.similarity_search_by_vector(v, k=4)
for r in results:
    print_ww(r.page_content)
    print('----')

[1.171875, 0.33398438, 0.3125, -0.24316406, 0.60546875, 0.41992188, -0.36132812, -6.580353e-05, 0.3203125, -0.66796875]
﻿What is Amazon SageMaker?: Is R supported with Amazon SageMaker?
Amazon SageMaker is a fully managed service to prepare data and build, train, and deploy machine
learning (ML) models for any use case with fully managed infrastructure, tools, and workflows.: Yes,
R is supported with Amazon SageMaker. You can use R within SageMaker notebook instances, which
include a preinstalled R kernel and the reticulate library. Reticulate offers an R interface for the
Amazon SageMaker Python SDK, enabling ML practitioners to build, train, tune, and deploy R models.
----
﻿What is Amazon SageMaker?: What is RStudio on Amazon SageMaker?
Amazon SageMaker is a fully managed service to prepare data and build, train, and deploy machine
learning (ML) models for any use case with fully managed infrastructure, tools, and workflows.:
RStudio on Amazon SageMaker is the first fully managed RSt

#### Memory
In any chatbot we will need a QA Chain with various options which are customized by the use case. But in a chatbot we will always need to keep the history of the conversation so the model can take it into consideration to provide the answer. In this example we use the [ConversationalRetrievalChain](https://python.langchain.com/docs/modules/chains/popular/chat_vector_db) from LangChain, together with a ConversationBufferMemory to keep the history of the conversation.

Source: https://python.langchain.com/docs/modules/chains/popular/chat_vector_db

Set `verbose` to `True` to see all the what is going on behind the scenes.

In [18]:
from langchain.chains.conversational_retrieval.prompts import CONDENSE_QUESTION_PROMPT

print_ww(CONDENSE_QUESTION_PROMPT.template)

Given the following conversation and a follow up question, rephrase the follow up question to be a
standalone question, in its original language.

Chat History:
{chat_history}
Follow Up Input: {question}
Standalone question:


#### Parameters used for ConversationRetrievalChain
* **retriever**: We used `VectorStoreRetriever`, which is backed by a `VectorStore`. To retrieve text, there are two search types you can choose: `"similarity"` or `"mmr"`. `search_type="similarity"` uses similarity search in the retriever object where it selects text chunk vectors that are most similar to the question vector.

* **memory**: Memory Chain to store the history 

* **condense_question_prompt**: Given a question from the user, we use the previous conversation and that question to make up a standalone question

* **chain_type**: If the chat history is long and doesn't fit the context you use this parameter and the options are `stuff`, `refine`, `map_reduce`, `map-rerank`

If the question asked is outside the scope of context, then the model will reply it doesn't know the answer

**Note**: if you are curious how the chain works, uncomment the `verbose=True` line.

In [19]:
# turn verbose to true to see the full logs and documents
from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory

memory_chain = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
qa = ConversationalRetrievalChain.from_llm(
    llm=cl_llm, 
    retriever=vectorstore_faiss_aws.as_retriever(), 
    memory=memory_chain,
    condense_question_prompt=CONDENSE_QUESTION_PROMPT,
    #verbose=True, 
    chain_type='stuff', # 'refine',
    #max_tokens_limit=300
)

Let's chat! ask the chatbot some questions about SageMaker, like:
1. What is SageMaker?
2. What is canvas?

In [20]:
chat = ChatUX(qa, retrievalChain=True)
chat.start_chat()

Starting chat bot


Output()

Your mileage might vary, but after 2 or 3 questions you will start to get some weird answers. In some cases, even in other languages.
This is happening for the same reasons outlined at the beginning of this notebook: the default langchain prompts are not optimal for Claude. 
In the following cell we are going to set two new prompts: one for the question rephrasing, and one to get the answer from that rephrased question.

In [21]:
# turn verbose to true to see the full logs and documents
from langchain.chains import ConversationalRetrievalChain
from langchain.schema import BaseMessage


# We are also providing a different chat history retriever which outputs the history as a Claude chat (ie including the \n\n)
_ROLE_MAP = {"human": "\n\nHuman: ", "ai": "\n\nAssistant: "}
def _get_chat_history(chat_history):
    buffer = ""
    for dialogue_turn in chat_history:
        if isinstance(dialogue_turn, BaseMessage):
            role_prefix = _ROLE_MAP.get(dialogue_turn.type, f"{dialogue_turn.type}: ")
            buffer += f"\n{role_prefix}{dialogue_turn.content}"
        elif isinstance(dialogue_turn, tuple):
            human = "\n\nHuman: " + dialogue_turn[0]
            ai = "\n\nAssistant: " + dialogue_turn[1]
            buffer += "\n" + "\n".join([human, ai])
        else:
            raise ValueError(
                f"Unsupported chat history format: {type(dialogue_turn)}."
                f" Full chat history: {chat_history} "
            )
    return buffer

# the condense prompt for Claude
condense_prompt_claude = PromptTemplate.from_template("""{chat_history}

Answer only with the new question.


Human: How would you ask the question considering the previous conversation: {question}


Assistant: Question:""")

# recreate the Claude LLM with more tokens to sample - this provides longer responses but introduces some latency
cl_llm = Bedrock(model_id="anthropic.claude-v2", client=boto3_bedrock, model_kwargs={"max_tokens_to_sample": 500})
memory_chain = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
qa = ConversationalRetrievalChain.from_llm(
    llm=cl_llm, 
    retriever=vectorstore_faiss_aws.as_retriever(), 
    #retriever=vectorstore_faiss_aws.as_retriever(search_type='similarity', search_kwargs={"k": 8}),
    memory=memory_chain,
    get_chat_history=_get_chat_history,
    #verbose=True,
    condense_question_prompt=condense_prompt_claude, 
    chain_type='stuff', # 'refine',
    #max_tokens_limit=300
)

# the LLMChain prompt to get the answer. the ConversationalRetrievalChange does not expose this parameter in the constructor
qa.combine_docs_chain.llm_chain.prompt = PromptTemplate.from_template("""
{context}

Human: Use at maximum 3 sentences to answer the question inside the <q></q> XML tags. 

<q>{question}</q>

Do not use any XML tags in the answer. If the answer is not in the context say "Sorry, I don't know as the answer was not found in the context"

Assistant:""")

Let's start another chat. Feel free to ask the following questions:

1. What is SageMaker?
2. what is canvas?

In [22]:
chat = ChatUX(qa, retrievalChain=True)
chat.start_chat()

Starting chat bot


Output()

#### Do some prompt engineering

You can "tune" your prompt to get more or less verbose answers. For example, try to change the number of sentences, or remove that instruction all-together. You might also need to change the number of `max_tokens_to_sample` (eg 1000 or 2000) to get the full answer.

### In this demo we used Claude LLM to create conversational interface with following patterns:

1. Chatbot (Basic - without context)

2. Chatbot using prompt template(Langchain)

3. Chatbot with personas

4. Chatbot with context